# 📚 Notebook d'Entraînement des Modèles de Prédiction du Churn

## 🎯 Objectif
Ce notebook implémente un pipeline complet de machine learning pour prédire le churn (désabonnement) des clients. Il couvre toutes les étapes essentielles :

1. **Chargement et exploration des données**
2. **Analyse exploratoire (EDA)**
3. **Prétraitement et feature engineering**
4. **Entraînement de plusieurs modèles**
5. **Optimisation des hyperparamètres**
6. **Évaluation et comparaison des modèles**
7. **Sélection et sauvegarde du meilleur modèle**

## 📊 Données
Le dataset contient des informations sur les clients d'une entreprise de télécommunications, incluant leurs caractéristiques démographiques, les services souscrits, et leur statut de churn.

## 🎓 Contexte Business
Le churn (taux d'attrition client) est un indicateur critique pour les entreprises. Perdre des clients coûte généralement plus cher que d'en acquérir de nouveaux. Ce modèle permet d'identifier proactivement les clients à risque pour mettre en place des actions de rétention ciblées.


---
## 1. Configuration et Imports

Cette section initialise l'environnement de travail en important toutes les bibliothèques nécessaires pour :
- La manipulation de données (pandas, numpy)
- La visualisation (matplotlib, seaborn)
- Le machine learning (scikit-learn)
- La persistance des modèles (joblib)
- Le logging des opérations


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import logging

# Prétraitement et modélisation
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Sauvegarde des modèles et des métriques
import joblib
import json

# Ignorer les warnings
import warnings
warnings.filterwarnings('ignore')


### 1.1 Organisation des Fichiers

Création d'une structure de dossiers pour organiser les artefacts du projet :
- **`data/`** : Données intermédiaires et transformées
- **`figures/`** : Graphiques et visualisations
- **`models/`** : Modèles entraînés (.pkl)
- **`metrics/`** : Métriques d'évaluation et rapports
- **`logs/`** : Logs d'exécution du projet


In [34]:
# Création des dossiers pour organiser les artefacts
folders = ['data', 'figures', 'models', 'metrics', 'logs']
for folder in folders:
    os.makedirs(folder, exist_ok=True)


### 1.2 Configuration du Logging

Le système de logging permet de tracer toutes les opérations importantes du pipeline. Les logs sont enregistrés dans `logs/project_logs.log` pour faciliter le debugging et l'audit des entraînements.


In [35]:
# Configuration du logger
logging.basicConfig(
    filename='logs/project_logs.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

logging.info('Initialisation du projet de prédiction du churn client.')


---
## 2. Chargement et Exploration des Données

### 2.1 Chargement des Données

Les données sont chargées depuis le fichier CSV et des aperçus sont générés :
- **`data.head()`** : Les 5 premières lignes pour inspection rapide
- **`data.info()`** : Types de données et valeurs manquantes
- **`data.describe()`** : Statistiques descriptives des variables numériques

Ces exports permettent une analyse rapide sans avoir à relancer le notebook.


In [36]:
# Chargement des données depuis un fichier CSV
data = pd.read_csv('../data/data.csv')

# Affichage des premières lignes pour un aperçu rapide
print("📊 Aperçu des données:")
print(data.head())
print(f"\n✅ Données chargées avec succès: {data.shape[0]} lignes et {data.shape[1]} colonnes")

# Enregistrement de l'aperçu des données
data.head().to_csv('data/data_preview.csv', index=False)
logging.info('Aperçu des données enregistré dans data/data_preview.csv')

# Informations sur le DataFrame
with open('data/data_info.txt', 'w') as f:
    data.info(buf=f)
logging.info('Informations sur les données enregistrées dans data/data_info.txt')

# Statistiques descriptives
print("\n📈 Statistiques descriptives des variables numériques:")
print(data.describe())

# Enregistrement des statistiques descriptives
data.describe().to_csv('data/data_describe.csv')
logging.info('Statistiques descriptives enregistrées dans data/data_describe.csv')

# Vérification des valeurs manquantes
print("\n🔍 Valeurs manquantes par colonne:")
missing_values = data.isnull().sum()
if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("Aucune valeur manquante détectée")


📊 Aperçu des données:
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovie

### 2.2 Nettoyage des Données

**Traitement de la variable `TotalCharges` :**
- Certaines valeurs peuvent être stockées comme texte au lieu de nombres
- Conversion en numérique avec gestion des erreurs (`errors='coerce'`)
- Imputation des valeurs manquantes par la médiane (robuste aux outliers)

**Encodage de la variable cible :**
- Transformation de `Churn` (Yes/No) en variable binaire (1/0)
- Facilite l'utilisation dans les algorithmes de machine learning


In [37]:
# Nettoyage de 'TotalCharges' si nécessaire
# Certaines valeurs peuvent être stockées comme string au lieu de numérique
print("🧹 Nettoyage de la colonne 'TotalCharges'...")

# Conversion en numérique avec gestion des erreurs
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# Affichage du nombre de valeurs manquantes créées
n_missing = data['TotalCharges'].isnull().sum()
if n_missing > 0:
    print(f"⚠️  {n_missing} valeurs non numériques converties en NaN")
    print(f"📊 Imputation par la médiane: {data['TotalCharges'].median():.2f}")
    
# Imputation des valeurs manquantes par la médiane (robuste aux outliers)
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# Encodage de la variable cible
data['Churn_encoded'] = data['Churn'].map({'Yes': 1, 'No': 0})

# Affichage de la distribution du churn
print(f"\n🎯 Distribution de la variable cible:")
print(f"  - Clients non-churned (No): {(data['Churn'] == 'No').sum()} ({(data['Churn'] == 'No').sum() / len(data) * 100:.1f}%)")
print(f"  - Clients churned (Yes): {(data['Churn'] == 'Yes').sum()} ({(data['Churn'] == 'Yes').sum() / len(data) * 100:.1f}%)")


🧹 Nettoyage de la colonne 'TotalCharges'...
⚠️  11 valeurs non numériques converties en NaN
📊 Imputation par la médiane: 1397.47

🎯 Distribution de la variable cible:
  - Clients non-churned (No): 5174 (73.5%)
  - Clients churned (Yes): 1869 (26.5%)


---
## 3. Analyse Exploratoire des Données (EDA)

L'EDA est une étape cruciale pour comprendre les données et identifier les patterns liés au churn.

### 3.1 Analyse des Variables Catégoriques

**Objectif :** Visualiser la distribution des variables catégoriques en fonction du statut de churn.

**Insights attendus :**
- Quels services ou caractéristiques sont associés à un taux de churn plus élevé ?
- Y a-t-il des déséquilibres importants dans certaines catégories ?
- Les contrats à long terme réduisent-ils le churn ?

Les barplots permettent de comparer visuellement les proportions de churn pour chaque catégorie.


In [38]:
# Liste des variables catégoriques
categorical_vars = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
                    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                    'Contract', 'PaperlessBilling', 'PaymentMethod']

# Paramètres pour les sous-graphiques
n_cols = 3
n_rows = int(np.ceil(len(categorical_vars) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for idx, var in enumerate(categorical_vars):
    sns.countplot(x=var, hue='Churn', data=data, ax=axes[idx])
    axes[idx].set_title(f'Distribution de {var} par Churn')
    axes[idx].set_xlabel(var)
    axes[idx].set_ylabel('Nombre')
    axes[idx].legend(title='Churn', loc='upper right')

# Supprimer les axes vides
for ax in axes[len(categorical_vars):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.savefig('figures/categorical_barplots.png')
plt.close()
logging.info('Barplots des variables catégoriques enregistrés dans figures/categorical_barplots.png')


### 3.2 Analyse des Variables Numériques

**Variables analysées :**
- **`tenure`** : Ancienneté du client (en mois) - Les clients récents sont-ils plus à risque ?
- **`MonthlyCharges`** : Frais mensuels - Un coût élevé augmente-t-il le churn ?
- **`TotalCharges`** : Charges totales cumulées - Corrélation avec la fidélité ?

Les boxplots révèlent les différences de distribution entre clients churned et non-churned.


In [39]:
# Variables numériques
numerical_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']

n_cols = 3
n_rows = int(np.ceil(len(numerical_vars) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten()

for idx, var in enumerate(numerical_vars):
    sns.boxplot(x='Churn', y=var, data=data, ax=axes[idx])
    axes[idx].set_title(f'Boxplot de {var} par Churn')
    axes[idx].set_xlabel('Churn')
    axes[idx].set_ylabel(var)

# Supprimer les axes vides
for ax in axes[len(numerical_vars):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.savefig('figures/numerical_boxplots.png')
plt.close()
logging.info('Boxplots des variables numériques enregistrés dans figures/numerical_boxplots.png')


### 3.3 Matrice de Corrélation

**Analyse des relations linéaires** entre les variables numériques et la variable cible.

**Points d'attention :**
- Multicolinéarité entre variables (corrélation > 0.7-0.8)
- Corrélation avec la variable cible (`Churn_encoded`)
- TotalCharges et tenure sont souvent fortement corrélés (logique : plus on reste, plus on paie)

Cette analyse guide les décisions de feature engineering.


In [40]:
# Matrice de corrélation
corr_matrix = data[numerical_vars + ['Churn_encoded']].corr()

# Heatmap de la matrice de corrélation
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Matrice de Corrélation')
plt.savefig('figures/correlation_matrix.png')
plt.close()
logging.info('Matrice de corrélation enregistrée dans figures/correlation_matrix.png')


---
## 4. Prétraitement et Feature Engineering

### 4.1 Séparation Features / Target

**Exclusions :**
- **`Churn`** : Variable cible (version texte)
- **`customerID`** : Identifiant unique, non informatif pour la prédiction
- **`Churn_encoded`** : Variable cible (version numérique)

**Catégorisation des variables :**
1. **Variables binaires** : 2 valeurs uniques (ex: gender, Partner)
2. **Variables multi-catégorielles** : Plus de 2 valeurs (ex: InternetService, Contract)
3. **Variables numériques** : Déjà identifiées (tenure, charges)

Cette catégorisation permet d'appliquer le bon type d'encodage à chaque type de variable.


In [41]:
# Séparation des features et de la cible
X = data.drop(['Churn', 'customerID', 'Churn_encoded'], axis=1)
y = data['Churn_encoded']

# Identification des variables binaires et catégoriques
binary_vars = [col for col in X.columns if X[col].nunique() == 2]
multi_category_vars = [col for col in X.columns if X[col].dtype == 'object' and col not in binary_vars]


### 4.2 Pipeline de Prétraitement

Utilisation de **`ColumnTransformer`** de scikit-learn pour appliquer différentes transformations en parallèle :

1. **Variables numériques → StandardScaler**
   - Centrage (moyenne = 0) et réduction (écart-type = 1)
   - Important pour les modèles sensibles à l'échelle (SVM, régression logistique)

2. **Variables binaires → OrdinalEncoder**
   - Transformation simple en 0/1
   - Préserve la nature binaire

3. **Variables catégorielles → OneHotEncoder**
   - Crée une colonne binaire par catégorie
   - `drop='first'` : évite la multicolinéarité parfaite
   - Exemple : InternetService → [DSL, Fiber optic] (No est la référence)

**Avantage :** Pipeline réutilisable et garantit la cohérence entre train/validation/test.


In [42]:
# Création du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_vars),
        ('bin', OrdinalEncoder(), binary_vars),
        ('cat', OneHotEncoder(drop='first'), multi_category_vars)
    ],
    remainder='drop'
)


### 4.3 Application du Prétraitement

Application du préprocesseur sur l'ensemble complet des features pour :
- Vérifier que la transformation fonctionne correctement
- Obtenir les noms des features après encodage
- Sauvegarder un aperçu des données transformées

**Note :** Dans le pipeline final, la transformation sera appliquée séparément sur train/val/test pour éviter le data leakage.


In [43]:
# Application du préprocesseur sur l'ensemble des données
X_normalized = preprocessor.fit_transform(X)

# Récupération des noms de colonnes après encodage
num_features = numerical_vars
bin_features = binary_vars
cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(multi_category_vars)
all_features = np.concatenate([num_features, bin_features, cat_features])

# Conversion en DataFrame
X_normalized_df = pd.DataFrame(X_normalized, columns=all_features)

# Enregistrement de la DataFrame normalisée
X_normalized_df.to_csv('data/X_normalized.csv', index=False)
logging.info('DataFrame normalisée enregistrée dans data/X_normalized.csv')


### 4.4 Division des Données (Train / Validation / Test)

**Stratégie de split :**
- **70% Train** : Entraînement des modèles
- **15% Validation** : Optimisation des hyperparamètres et sélection du modèle
- **15% Test** : Évaluation finale du modèle sélectionné (non utilisé dans ce notebook)

**`stratify=y`** : Préserve la proportion de churn dans chaque ensemble (important car le churn est souvent déséquilibré).

**Pourquoi 3 ensembles ?**
- Évite l'overfitting sur les données de validation
- Le test set donne une estimation non biaisée des performances en production


In [44]:
# Division initiale en entraînement et temporaire (70% entraînement, 30% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Division du temporaire en validation et test (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Affichage des tailles et distribution
print("📊 Division des données:")
print(f"  - Entraînement: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Validation: {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n🎯 Distribution du churn:")
print(f"  - Train: {y_train.sum()}/{len(y_train)} churned ({y_train.mean()*100:.1f}%)")
print(f"  - Validation: {y_val.sum()}/{len(y_val)} churned ({y_val.mean()*100:.1f}%)")
print(f"  - Test: {y_test.sum()}/{len(y_test)} churned ({y_test.mean()*100:.1f}%)")

# Logging
logging.info(f"Taille de l'entraînement: {X_train.shape}")
logging.info(f"Taille de la validation: {X_val.shape}")
logging.info(f"Taille du test: {X_test.shape}")


📊 Division des données:
  - Entraînement: 4930 samples (70.0%)
  - Validation: 1056 samples (15.0%)
  - Test: 1057 samples (15.0%)

🎯 Distribution du churn:
  - Train: 1308/4930 churned (26.5%)
  - Validation: 280/1056 churned (26.5%)
  - Test: 281/1057 churned (26.6%)


---
## 5. Entraînement des Modèles

### 5.1 Création des Pipelines

Chaque pipeline combine le prétraitement et un algorithme de machine learning :

**1. Régression Logistique**
- Modèle linéaire simple et interprétable
- `max_iter=1000` : Augmente les itérations pour convergence
- Baseline solide pour problèmes de classification binaire

**2. Random Forest**
- Ensemble de arbres de décision
- Gère bien les non-linéarités et interactions
- Moins sensible aux outliers

**3. Support Vector Machine (SVM)**
- Trouve la frontière optimale entre classes
- `probability=True` : Active les probabilités (nécessaire pour ROC-AUC)
- Peut capturer des patterns complexes avec les noyaux

**Avantage des pipelines :** Garantit que le prétraitement est appliqué de manière identique à chaque étape.


In [45]:
# Pipelines pour les modèles sans hyperparamètres
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(probability=True, random_state=42))
])

logging.info('Pipelines scikit-learn créés pour chaque modèle.')


### 5.2 Grilles d'Hyperparamètres

Définition des hyperparamètres à tester pour chaque modèle via GridSearchCV.

**Régression Logistique :**
- **`C`** : Inverse de la régularisation (↑C = ↓régularisation = modèle plus complexe)
- **`penalty='l2'`** : Régularisation Ridge (pénalise les coefficients élevés)

**Random Forest :**
- **`n_estimators`** : Nombre d'arbres (↑ = plus stable mais plus lent)
- **`max_depth`** : Profondeur maximale (contrôle l'overfitting)
- **`min_samples_split/leaf`** : Critères d'arrêt pour éviter l'overfitting

**SVM :**
- **`C`** : Contrôle le compromis biais/variance
- **`kernel`** : Type de frontière (linear = linéaire, rbf = non-linéaire)
- **`gamma`** : Influence du kernel RBF (↑gamma = plus de complexité)

**Note :** GridSearchCV testera toutes les combinaisons pour trouver la meilleure.


In [46]:
# Paramètres pour la Régression Logistique
param_grid_lr = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2'],
    'classifier__solver': ['lbfgs']
}

# Paramètres pour la Forêt Aléatoire
param_grid_rf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

# Paramètres pour le SVM
param_grid_svc = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['rbf', 'linear'],
    'classifier__gamma': ['scale', 'auto']
}


### 5.3 Fonction d'Optimisation

**GridSearchCV** effectue :
1. **Validation croisée 5-fold** : Divise train en 5 parties, entraîne sur 4, valide sur 1 (rotation)
2. **Optimisation sur ROC-AUC** : Métrique robuste pour données déséquilibrées
3. **Parallélisation** (`n_jobs=-1`) : Utilise tous les CPU disponibles

**Pourquoi ROC-AUC ?**
- Insensible au déséquilibre des classes
- Évalue la qualité du ranking des prédictions
- AUC=0.5 : modèle aléatoire / AUC=1.0 : modèle parfait

La fonction retourne le meilleur estimateur (modèle + hyperparamètres optimaux).


In [47]:
def fine_tune_model(pipeline, param_grid, X_train, y_train, model_name):
    logging.info(f"Début du fine-tuning pour {model_name}")
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    logging.info(f"Meilleurs paramètres pour {model_name}: {grid_search.best_params_}")
    return grid_search.best_estimator_


### 5.4 Entraînement et Optimisation

Lancement du fine-tuning pour les 3 modèles en parallèle.

**Processus :**
- Pour chaque combinaison d'hyperparamètres
- GridSearch entraîne le modèle 5 fois (5-fold CV)
- Calcule le ROC-AUC moyen
- Sélectionne la meilleure configuration

**Nombre total d'entraînements :**
- Logistic Regression : 4 combinaisons × 5 folds = 20 fits
- Random Forest : 24 combinaisons × 5 folds = 120 fits  
- SVM : 12 combinaisons × 5 folds = 60 fits

⏱️ **Note :** Cette cellule peut prendre plusieurs minutes selon la puissance de calcul.


In [48]:
# Fine-tuning pour la Régression Logistique
best_model_lr = fine_tune_model(pipeline_lr, param_grid_lr, X_train, y_train, 'LogisticRegression')

# Fine-tuning pour la Forêt Aléatoire
best_model_rf = fine_tune_model(pipeline_rf, param_grid_rf, X_train, y_train, 'RandomForest')

# Fine-tuning pour le SVM
best_model_svc = fine_tune_model(pipeline_svc, param_grid_svc, X_train, y_train, 'SVC')


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


---
## 6. Évaluation des Modèles

### 6.1 Fonction de Visualisation de la Matrice de Confusion

La matrice de confusion affiche :
- **Vrais Négatifs (TN)** : Clients correctement prédits comme non-churned
- **Faux Positifs (FP)** : Clients non-churned prédits comme churned (Type I)
- **Faux Négatifs (FN)** : Clients churned prédits comme non-churned (Type II - ⚠️ critique !)
- **Vrais Positifs (TP)** : Clients correctement prédits comme churned

Les pourcentages facilitent l'interprétation des performances par classe.


In [49]:
def plot_confusion_matrix(cm, class_names, title='Confusion matrix', cmap=plt.cm.Blues, file_name='confusion_matrix.png'):
    """
    Trace la matrice de confusion personnalisée.

    Args:
        cm (np.ndarray): Matrice de confusion.
        class_names (list): Liste des noms des classes.
        title (str): Titre du graphique.
        cmap (matplotlib.colors.Colormap): Colormap pour le graphique.
        file_name (str): Nom du fichier pour enregistrer le graphique.
    """
    logging.info(f"Plotting confusion matrix for {title}")
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        percentage = 100 * cm[i, j] / cm[i, :].sum() if cm[i, :].sum() != 0 else 0
        plt.text(j, i, f'{cm[i, j]} ({percentage:.2f}%)',
                 horizontalalignment="center", verticalalignment='center',
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(file_name)
    plt.close()


### 6.2 Fonction d'Évaluation Complète

Pour chaque modèle, calcule et sauvegarde :

**1. Métriques de classification**
- **Précision (Precision)** : Parmi les prédictions positives, combien sont correctes ?
  - Important si le coût d'un FP est élevé
- **Rappel (Recall)** : Parmi les vrais positifs, combien sont détectés ?
  - Critique pour le churn : ne pas rater de clients à risque
- **F1-Score** : Moyenne harmonique de précision et rappel
  - Équilibre entre les deux

**2. Courbe ROC et AUC**
- Visualise le compromis entre TPR (True Positive Rate) et FPR (False Positive Rate)
- AUC résume la performance en un seul chiffre

**3. Sauvegarde des artefacts**
- Rapports texte pour revue détaillée
- Graphiques pour présentation
- Métriques JSON pour tracking MLOps


In [50]:
def evaluate_model(model, X_val, y_val, model_name):
    logging.info(f'Évaluation du modèle {model_name}')
    
    # Prédictions
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]
    
    # Calcul des métriques
    report = classification_report(y_val, y_pred, output_dict=True)
    auc = roc_auc_score(y_val, y_proba)
    fpr, tpr, thresholds = roc_curve(y_val, y_proba)
    
    # Enregistrement du rapport de classification
    with open(f'metrics/classification_report_{model_name}.txt', 'w') as f:
        f.write(classification_report(y_val, y_pred))
    logging.info(f'Rapport de classification enregistré pour {model_name}')
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_pred)
    class_names = ['No', 'Yes']
    plot_confusion_matrix(cm, class_names, title=f'Matrice de Confusion - {model_name}', file_name=f'figures/confusion_matrix_{model_name}.png')
    
    # Courbe ROC
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'Courbe ROC - {model_name}')
    plt.xlabel('Taux de Faux Positifs')
    plt.ylabel('Taux de Vrais Positifs')
    plt.legend(loc='lower right')
    plt.savefig(f'figures/roc_curve_{model_name}.png')
    plt.close()
    logging.info(f'Courbe ROC enregistrée pour {model_name}')
    
    # Sauvegarder les métriques
    metrics = {
        'classification_report': report,
        'auc_roc': auc
    }
    with open(f'metrics/metrics_{model_name}.json', 'w') as f:
        json.dump(metrics, f, indent=4)
    logging.info(f'Métriques enregistrées pour {model_name}')
    
    return report, auc


### 6.3 Évaluation sur l'Ensemble de Validation

Application de la fonction d'évaluation aux 3 modèles optimisés.

Les résultats sont stockés dans :
- `models_reports` : Rapports de classification détaillés
- `models_auc` : Scores AUC-ROC pour chaque modèle

Ces métriques permettront de comparer objectivement les modèles et de sélectionner le meilleur.


In [51]:
# Dictionnaires pour stocker les rapports et les AUC
models_reports = {}
models_auc = {}

# Évaluation du modèle Logistic Regression
report_lr, auc_lr = evaluate_model(best_model_lr, X_val, y_val, 'LogisticRegression')
models_reports['LogisticRegression'] = report_lr
models_auc['LogisticRegression'] = auc_lr

# Évaluation du modèle Random Forest
report_rf, auc_rf = evaluate_model(best_model_rf, X_val, y_val, 'RandomForest')
models_reports['RandomForest'] = report_rf
models_auc['RandomForest'] = auc_rf

# Évaluation du modèle SVC
report_svc, auc_svc = evaluate_model(best_model_svc, X_val, y_val, 'SVC')
models_reports['SVC'] = report_svc
models_auc['SVC'] = auc_svc


### 6.4 Sauvegarde des Modèles

Persistance des 3 modèles optimisés avec `joblib` :
- Format efficace pour les objets Python/scikit-learn
- Permet le chargement rapide pour l'inférence
- Inclut le pipeline complet (prétraitement + modèle)

Ces fichiers `.pkl` peuvent être déployés directement en production.


In [52]:
# Sauvegarde des modèles optimisés
print("💾 Sauvegarde des modèles...")

joblib.dump(best_model_lr, 'models/model_LogisticRegression.pkl')
print("  ✅ Logistic Regression sauvegardé")

joblib.dump(best_model_rf, 'models/model_RandomForest.pkl')
print("  ✅ Random Forest sauvegardé")

joblib.dump(best_model_svc, 'models/model_SVC.pkl')
print("  ✅ SVM sauvegardé")

logging.info('Modèles optimisés sauvegardés dans le dossier models.')
print("\n✨ Tous les modèles ont été sauvegardés avec succès dans le dossier 'models/'")


💾 Sauvegarde des modèles...
  ✅ Logistic Regression sauvegardé
  ✅ Random Forest sauvegardé
  ✅ SVM sauvegardé

✨ Tous les modèles ont été sauvegardés avec succès dans le dossier 'models/'


---
## 7. Comparaison et Sélection du Meilleur Modèle

### 7.1 Consolidation des Rapports

Agrégation de tous les rapports de classification dans un DataFrame unique.

Facilite la comparaison côte à côte des performances de chaque modèle sur toutes les métriques.


In [53]:
# Création d'une liste pour stocker les rapports
classification_reports = []

for model_name, report in models_reports.items():
    report_df = pd.DataFrame(report).transpose()
    report_df['model'] = model_name
    report_df['metric'] = report_df.index
    classification_reports.append(report_df)

# Concaténer tous les rapports
all_reports_df = pd.concat(classification_reports, axis=0)

# Réorganisation des colonnes
cols = ['model', 'metric'] + [col for col in all_reports_df.columns if col not in ['model', 'metric']]
all_reports_df = all_reports_df[cols]

# Enregistrement dans un fichier CSV
all_reports_df.to_csv('metrics/all_classification_reports.csv', index=False)
logging.info('Tous les rapports de classification enregistrés dans metrics/all_classification_reports.csv')


### 7.2 Benchmarking et Sélection

**Méthode de scoring pondéré :**
- Chaque métrique reçoit un poids égal (25%)
- Score final = moyenne pondérée des 4 métriques clés

**⚠️ Note :** Dans un contexte business réel, les poids devraient refléter les priorités métier :
- Si le coût de manquer un churner est élevé → augmenter le poids du recall
- Si les actions de rétention sont coûteuses → privilégier la precision

**Classe positive ('1') :** Le scoring se base uniquement sur la classe churn, car c'est la classe d'intérêt.

Le modèle avec le score final le plus élevé est sélectionné comme meilleur modèle.


In [54]:
# Poids des métriques pour le scoring final
# Ajustez ces poids selon vos priorités business
weights = {
    'precision': 0.25,  # Importance de minimiser les faux positifs
    'recall': 0.25,     # Importance de détecter tous les churners
    'f1-score': 0.25,   # Équilibre entre précision et rappel
    'auc_roc': 0.25     # Performance globale du classifieur
}

benchmark_results = {}

print("📊 Calcul des scores finaux pondérés...\n")

for model_name, report in models_reports.items():
    # Récupérer les scores pour la classe positive ('1')
    metrics_class_1 = report['1']
    auc = models_auc[model_name]
    
    # Calcul du score final pondéré
    final_score = (
        weights['precision'] * metrics_class_1['precision'] +
        weights['recall'] * metrics_class_1['recall'] +
        weights['f1-score'] * metrics_class_1['f1-score'] +
        weights['auc_roc'] * auc
    )
    
    benchmark_results[model_name] = {
        'precision': metrics_class_1['precision'],
        'recall': metrics_class_1['recall'],
        'f1-score': metrics_class_1['f1-score'],
        'auc_roc': auc,
        'final_score': final_score
    }
    
    # Affichage des résultats
    print(f"{model_name}:")
    print(f"  Precision: {metrics_class_1['precision']:.4f}")
    print(f"  Recall: {metrics_class_1['recall']:.4f}")
    print(f"  F1-Score: {metrics_class_1['f1-score']:.4f}")
    print(f"  AUC-ROC: {auc:.4f}")
    print(f"  📈 Score Final: {final_score:.4f}\n")

# Création de la DataFrame des résultats
benchmark_df = pd.DataFrame(benchmark_results).T

# Enregistrement de la DataFrame des scores
benchmark_df.to_csv('metrics/benchmark_results.csv')
logging.info('Résultats du benchmarking enregistrés dans metrics/benchmark_results.csv')

# Affichage du meilleur modèle
best_model_name = benchmark_df['final_score'].idxmax()
best_score = benchmark_df.loc[best_model_name, 'final_score']
print("=" * 60)
print(f"🏆 MEILLEUR MODÈLE: {best_model_name}")
print(f"📊 Score Final: {best_score:.4f}")
print("=" * 60)

logging.info(f'Le meilleur modèle est : {best_model_name} (score: {best_score:.4f})')


📊 Calcul des scores finaux pondérés...

LogisticRegression:
  Precision: 0.6471
  Recall: 0.5893
  F1-Score: 0.6168
  AUC-ROC: 0.8452
  📈 Score Final: 0.6746

RandomForest:
  Precision: 0.6364
  Recall: 0.5250
  F1-Score: 0.5753
  AUC-ROC: 0.8423
  📈 Score Final: 0.6448

SVC:
  Precision: 0.6403
  Recall: 0.5786
  F1-Score: 0.6079
  AUC-ROC: 0.8375
  📈 Score Final: 0.6661

🏆 MEILLEUR MODÈLE: LogisticRegression
📊 Score Final: 0.6746


### 7.3 Visualisations Comparatives

**1. Comparaison multi-métriques**
- Barplot groupé montrant les 4 métriques pour chaque modèle
- Permet d'identifier visuellement les forces/faiblesses de chaque approche

**2. Score final pondéré**
- Barplot simple du score agrégé
- Conclusion visuelle pour la sélection du modèle

Ces visualisations facilitent la communication des résultats aux parties prenantes non-techniques.


In [55]:
# Comparaison des modèles sur les métriques
benchmark_df[['precision', 'recall', 'f1-score', 'auc_roc']].plot(kind='bar', figsize=(10, 6))
plt.title('Comparaison des Modèles sur les Différentes Métriques')
plt.ylabel('Score')
plt.xlabel('Modèle')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('figures/model_comparison_metrics.png')
plt.close()
logging.info('Figure de comparaison des métriques enregistrée dans figures/model_comparison_metrics.png')

# Figure pour le score final
benchmark_df['final_score'].plot(kind='bar', figsize=(8, 6))
plt.title('Score Final Pondéré des Modèles')
plt.ylabel('Score Final')
plt.xlabel('Modèle')
plt.tight_layout()
plt.savefig('figures/model_final_scores.png')
plt.close()
logging.info('Figure des scores finaux enregistrée dans figures/model_final_scores.png')


---
## 8. Conclusion et Prochaines Étapes

### ✅ Réalisations
- Pipeline ML complet de bout en bout
- 3 modèles entraînés et optimisés
- Évaluation rigoureuse sur ensemble de validation
- Sélection objective du meilleur modèle
- Artefacts sauvegardés pour déploiement

### 🔄 Prochaines Étapes

**1. Validation finale**
- Évaluer le modèle sélectionné sur le test set (15% des données non vues)
- Vérifier qu'il n'y a pas d'overfitting

**2. Analyse d'erreurs**
- Identifier les patterns de faux négatifs (clients churned non détectés)
- Analyser les caractéristiques des faux positifs

**3. Feature importance**
- Pour Random Forest : analyser l'importance des features
- Pour Logistic Regression : interpréter les coefficients

**4. Amélioration du modèle**
- Feature engineering avancé
- Gestion du déséquilibre des classes (SMOTE, class weights)
- Test d'algorithmes additionnels (XGBoost, LightGBM)

**5. Déploiement**
- Créer une API REST avec le modèle
- Intégrer dans l'application Streamlit (déjà fait)
- Mise en place du monitoring en production

### 📚 Ressources
- Documentation complète dans le dossier `docs/`
- Guides de bonnes pratiques ML et déploiement
- Application Streamlit pour l'inférence interactive

---
**📧 Questions ou suggestions ?** Consultez le README.md ou ouvrez une issue sur GitHub.
